In [36]:
# 查看是否安装“arules”
# 已安装
which(installed.packages() == "arules")

[1] 1

In [42]:
# 查看GoodsOrder数据集
file.show("../input/data/task/GoodsOrder.csv")

In [41]:
# 查看Goodtype数据集
file.show("../input/data/task/GoodsTypes.csv", encoding = "gb2312")

In [50]:
help(read.csv)

In [94]:
# 载入数据集
order_data <- read.csv("../input/data/task/GoodsOrder.csv", stringsAsFactor = F)
head(order_data)

# 修改编码为gb2312
type_data <- read.csv("../input/data/task/GoodsTypes.csv", stringsAsFactor = F, fileEncoding = 'gb2312')
head(type_data)

# 合并数据集
goods <- merge(order_data["Goods"], type_data, key = "Goods")
head(goods)

,ID,Goods
,<int>,<chr>
1,1,柑橘类水果
2,1,人造黄油
3,1,即食汤
4,1,半成品面包
5,2,咖啡
6,2,热带水果


,Goods,Types
,<chr>,<chr>
1,白饭,熟食
2,白酒,酒精饮料
3,白兰地,酒精饮料
4,白面包,西点
5,半成品面包,西点
6,包装袋,百货


,Goods,Types
,<chr>,<chr>
1,白饭,熟食
2,白饭,熟食
3,白饭,熟食
4,白饭,熟食
5,白饭,熟食
6,白饭,熟食


In [87]:
# 统计热销商品
hot_goods <- data.frame(table(order_data[, 2]))
# 改列名
names(hot_goods) <- c("goods", "count")
# 排序
hot_goods <- hot_goods[order(hot_goods$count, decreasing = T), ]
# 百分比
hot_goods["percent"] <- hot_goods$count / sum(hot_goods$count)
head(hot_goods)

# 保存热销商品的文件
write.csv(hot_goods, "../output/file/hot_goods.csv", row.names = F)
# 显示保存情况
file.show("../output/file/hot_goods.csv")

,goods,count,percent
,<fct>,<int>,<dbl>
110,全脂牛奶,2513,0.05794729
103,其他蔬菜,1903,0.04388129
87,面包卷,1809,0.04171375
121,苏打,1715,0.03954620
123,酸奶,1372,0.03163696
100,瓶装水,1087,0.02506514


In [104]:
# 统计热销种类
hot_type <- data.frame(table(type_data[, 2]))
# 改列名
names(hot_type) <- c("goods", "count")
# 百分比
hot_type["percent"] <- hot_type$count / sum(hot_type$count)
# 排序
hot_type <- hot_type[order(hot_type$count, decreasing = T), ]
head(hot_type)

# 写文件
write.csv(hot_type, "../output/file/hot_type.csv", row.names = F)

# 显示写入的文件
file.show("../output/file/hot_goods.csv")

,goods,count,percent
,<fct>,<int>,<dbl>
1,百货,43,0.25443787
6,米粮调料,32,0.18934911
10,西点,21,0.12426036
7,肉类,16,0.09467456
3,果蔬,14,0.08284024
2,非酒精饮料,11,0.06508876


In [109]:
head(goods)

,Goods,Types
,<chr>,<chr>
1,白饭,熟食
2,白饭,熟食
3,白饭,熟食
4,白饭,熟食
5,白饭,熟食
6,白饭,熟食


In [123]:
# 非酒精
drink <- goods[goods$Types == "非酒精饮料", ]
# 统计
hot_drink <- data.frame(table(drink$Goods)) 
# 改列名
names(hot_drink) <- c("goods", "count")
# 排序
hot_drink <- hot_drink[order(hot_drink$count, decreasing = T), ]
# 百分比
hot_drink["persent"] <- hot_drink$count / sum(hot_drink$count)

# 写文件
write.csv(hot_drink, "../output/file/hot_drink.csv", row.names = F)
# 显示文件
file.show("../output/file/hot_drink.csv")

In [126]:
GoodsOrder <- read.csv("../input/data/task/GoodsOrder.csv", stringsAsFactors = FALSE)
library(arules)  # 导入所需库包

# 数据形式转换
dataList <- list()
for (i in unique(GoodsOrder$ID)) {
    dataList[[i]] <- GoodsOrder[which(GoodsOrder$ID == i), 2]
}

In [131]:
head(dataList)

[[1]]
[1] "柑橘类水果" "人造黄油"   "即食汤"     "半成品面包"

[[2]]
[1] "咖啡"     "热带水果" "酸奶"    

[[3]]
[1] "全脂牛奶"

[[4]]
[1] "奶油乳酪"   "肉泥"       "仁果类水果" "酸奶"      

[[5]]
[1] "炼乳"     "长面包"   "其他蔬菜" "全脂牛奶"

[[6]]
[1] "腐蚀性清洁剂" "黄油"         "白饭"         "全脂牛奶"     "酸奶"

In [141]:
# 转化为transactions
TransRep <- as(dataList, "transactions")

In [139]:
# 查看item
inspect(TransRep[1:10])

     items                                             
[1]  {半成品面包,柑橘类水果,即食汤,人造黄油}           
[2]  {咖啡,热带水果,酸奶}                              
[3]  {全脂牛奶}                                        
[4]  {奶油乳酪,仁果类水果,肉泥,酸奶}                   
[5]  {炼乳,其他蔬菜,全脂牛奶,长面包}                   
[6]  {白饭,腐蚀性清洁剂,黄油,全脂牛奶,酸奶}            
[7]  {面包卷}                                          
[8]  {超高温杀菌的牛奶,开胃酒,面包卷,瓶装啤酒,其他蔬菜}
[9]  {盆栽}                                            
[10] {谷物,全脂牛奶}                                   


In [142]:
RulesRep <- apriori(TransRep, parameter = list(support = 0.02, confidence = 0.25))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
       0.25    0.1    1 none FALSE            TRUE       5    0.02      1
 maxlen target   ext
     10  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 196 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[169 item(s), 9835 transaction(s)] done [0.00s].
sorting and recoding items ... [59 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3 done [0.00s].
writing ... [50 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].


In [143]:
inspect(RulesRep[1:10])

     lhs             rhs        support    confidence lift      count
[1]  {}           => {全脂牛奶} 0.25551601 0.2555160  1.0000000 2513 
[2]  {冷冻蔬菜}   => {全脂牛奶} 0.02043721 0.4249471  1.6630940  201 
[3]  {牛肉}       => {全脂牛奶} 0.02125064 0.4050388  1.5851795  209 
[4]  {凝乳}       => {全脂牛奶} 0.02613116 0.4904580  1.9194805  257 
[5]  {猪肉}       => {其他蔬菜} 0.02165735 0.3756614  1.9414764  213 
[6]  {猪肉}       => {全脂牛奶} 0.02216573 0.3844797  1.5047187  218 
[7]  {冷冻热狗肠} => {全脂牛奶} 0.02053889 0.3482759  1.3630295  202 
[8]  {瓶装啤酒}   => {全脂牛奶} 0.02043721 0.2537879  0.9932367  201 
[9]  {黑面包}     => {全脂牛奶} 0.02521607 0.3887147  1.5212930  248 
[10] {人造黄油}   => {全脂牛奶} 0.02419929 0.4131944  1.6170980  238 


In [144]:
# 排序
inspect(sort(RulesRep, by = "lift")[1:25]) 

     lhs                      rhs          support    confidence lift     count
[1]  {其他蔬菜,全脂牛奶}   => {根茎类蔬菜} 0.02318251 0.3097826  2.842082 228  
[2]  {仁果类水果}          => {热带水果}   0.02043721 0.2701613  2.574648 201  
[3]  {根茎类蔬菜,全脂牛奶} => {其他蔬菜}   0.02318251 0.4740125  2.449770 228  
[4]  {根茎类蔬菜}          => {其他蔬菜}   0.04738180 0.4347015  2.246605 466  
[5]  {其他蔬菜,全脂牛奶}   => {酸奶}       0.02226741 0.2975543  2.132979 219  
[6]  {酸奶油}              => {其他蔬菜}   0.02887646 0.4028369  2.081924 284  
[7]  {酸奶油}              => {酸奶}       0.02074225 0.2893617  2.074251 204  
[8]  {全脂牛奶,酸奶}       => {其他蔬菜}   0.02226741 0.3974592  2.054131 219  
[9]  {其他蔬菜,酸奶}       => {全脂牛奶}   0.02226741 0.5128806  2.007235 219  
[10] {热带水果}            => {酸奶}       0.02928317 0.2790698  2.000475 288  
[11] {黄油}                => {全脂牛奶}   0.02755465 0.4972477  1.946053 271  
[12] {猪肉}                => {其他蔬菜}   0.02165735 0.3756614  1.941476 213  
[13] {凝乳}                => {全脂牛奶}   0.02613116 0.4904580  1.919

In [152]:
file.show("../input/data/example/bread.csv", encoding = "gb2312")

In [149]:
file.show("../input/data/example/Breakfast.csv", encoding = "gb2312")

In [151]:
file.show("../input/data/example/drink.csv", encoding = "gb2312")

In [154]:
bread_data <- read.csv("../input/data/example/bread.csv", fileEncoding = "gb2312", stringsAsFactor = F)
breakfast_data <- read.csv("../input/data/example/Breakfast.csv", fileEncoding = "gb2312", stringsAsFactor = F)
drink_data <- read.csv("../input/data/example/drink.csv", fileEncoding = "gb2312", stringsAsFactor = F)

In [170]:
names(bread_data) = c("Goods", "sort")
# 合并数据
breakfast <- merge(bread_data, breakfast_data["Goods"], key = "Goods")
# 缺失值检测
anyNA(breakfast)
head(breakfast)

[1] FALSE

,Goods,sort
,<chr>,<chr>
1,蛋挞,点心面包
2,蛋挞,点心面包
3,蛋挞,点心面包
4,蛋挞,点心面包
5,蛋挞,点心面包
6,蛋挞,点心面包


In [177]:
# 统计
hot_breakfast <- data.frame(table(breakfast$Goods)) 
# 改列名
names(hot_breakfast) <- c("goods", "count")
# 排序
hot_breakfast <- hot_breakfast[order(hot_breakfast$count, decreasing = T), ]
# 百分比
hot_breakfast["percent"] <- hot_breakfast$count / sum(hot_breakfast$count)

# 写文件
write.csv(hot_breakfast, "../output/file/hot_breakfast.csv", row.names = F)
# 显示文件
file.show("../output/file/hot_breakfast.csv")

In [181]:
library(arules)  # 导入所需库包

# 数据形式转换
dataList <- list()
for (i in unique(breakfast_data$ID)) {
    dataList[[i]] <- breakfast_data[which(breakfast_data$ID == i), 2]
}

In [182]:
head(dataList)

[[1]]
[1] "提子土司" "纯牛奶"  

[[2]]
[1] "提拉米苏" "三明治"   "纯牛奶"  

[[3]]
[1] "蒜香芝士面包" "肉松面包"     "豆浆"        

[[4]]
[1] "三明治"     "汉堡包"     "黑森林蛋糕" "奶茶"      

[[5]]
[1] "小麦土司"

[[6]]
[1] "芝士蛋糕"     "黑森林蛋糕"   "红豆酥皮面包"

In [183]:
# 转化为transactions
TransRep <- as(dataList, "transactions")
# 查看item
inspect(TransRep[1:10])

     items                                   
[1]  {纯牛奶,提子土司}                       
[2]  {纯牛奶,三明治,提拉米苏}                
[3]  {豆浆,肉松面包,蒜香芝士面包}            
[4]  {汉堡包,黑森林蛋糕,奶茶,三明治}         
[5]  {小麦土司}                              
[6]  {黑森林蛋糕,红豆酥皮面包,芝士蛋糕}      
[7]  {酥皮面包}                              
[8]  {纯牛奶,蛋挞,豆奶}                      
[9]  {咖啡,热狗}                             
[10] {豆奶,汉堡包,奶茶,蒜香芝士面包,提子土司}


In [186]:
RulesRep <- apriori(TransRep, parameter = list(support = 0.02, confidence = 0.4))
# 排序
inspect(sort(RulesRep, by = "lift")[1:25]) 

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.4    0.1    1 none FALSE            TRUE       5    0.02      1
 maxlen target   ext
     10  rules FALSE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 1 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[24 item(s), 60 transaction(s)] done [0.00s].
sorting and recoding items ... [24 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3 4 done [0.00s].
writing ... [76 rule(s)] done [0.00s].
creating S4 object  ... done [0.00s].
     lhs                             rhs            support    confidence
[1]  {纯牛奶,豆浆}                => {芝士蛋糕}     0.03333333 0.6666667 
[2]  {蒜香芝士面包,提子土司}      => {汉堡包}       0.03333333 1.0000000 
[3]  {汉堡包,提子土司}            => {蒜香芝士面包} 0.03333333 1.0000000 
[4]  {汉堡包,蒜香芝士面包}        => 

In [187]:
which(installed.packages() == "arulesViz")

integer(0)

In [ ]:
install.packages("arulesViz")

Installing package into ‘/home/jianli/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)

Warning message:
“dependencies ‘TSP’, ‘caTools’ are not available”
also installing the dependencies ‘gridExtra’, ‘viridis’, ‘gtools’, ‘gdata’, ‘tidyselect’, ‘plogr’, ‘qap’, ‘gclus’, ‘dendextend’, ‘gplots’, ‘registry’, ‘tidyr’, ‘hexbin’, ‘dplyr’, ‘data.table’, ‘scatterplot3d’, ‘seriation’, ‘igraph’, ‘plotly’, ‘visNetwork’


